In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from dotenv import load_dotenv
import os
from IPython.display import Markdown, display
from openai import OpenAI
import locale
import numpy as np
from sklearn.metrics import accuracy_score
import utils.extractors as extractors
import utils.millers as millers
import gradio as gr
from datetime import datetime
from functools import partial
from sqlalchemy import text

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Load environment variables from .env
load_dotenv()

# Data Extraction

## Categories

In [ ]:
df_cat = extractors.fetch_categories()
df_cat

## Transactions

In [ ]:
df_trans = extractors.fetch_sample_transactions()
df_trans

In [ ]:
transaction = df_trans.iloc[0]

labeler = millers.SimpleOpenAILabeler(transaction)
subcat = labeler.get_category()

parent_name = df_cat[df_cat.category_name == subcat].parent_name.values[0]

transaction, parent_name, subcat

In [ ]:
# display(Markdown(labeler.messages[0]['content']))
labeler.messages[1]

# Dashboard

In [ ]:
def get_subcategories(parent):
    filtered = df_cat[df_cat.parent_name == parent].category_name.unique().tolist()
    return gr.Dropdown(choices=filtered, interactive=True, label='Sub Category')

def openai_call(transaction_id):
    transaction = df_trans.loc[transaction_id]

    labeler = millers.SimpleOpenAILabeler(transaction)
    category_name = labeler.get_category()

    parent_name = df_cat[df_cat.category_name == category_name].parent_name.values[0]

    return parent_name, category_name

def submit_cat(trans_id, subcategory):
    df_trans.loc[trans_id, 'category_name'] = subcategory
    return True

def fetch_offset_transaction(offset, trans_id):
    new_idx = df_trans.index.get_loc(trans_id) + offset

    if new_idx < 0 or new_idx >= len(df_trans):
        raise gr.Error('No more of those crazy transactions!')

    new_transaction = df_trans.iloc[new_idx]

    trans_id = int(new_transaction.name)
    trans_date = datetime.combine(new_transaction['transaction_date'], datetime.min.time())
    trans_desc = new_transaction['description']
    trans_amount = new_transaction['amount']
    submit_box = new_transaction['category_name'] is not None

    return trans_id, trans_date, trans_desc, trans_amount, submit_box

fetch_prev_transaction = partial(fetch_offset_transaction, -1)
fetch_next_transaction = partial(fetch_offset_transaction,  1)

transaction = df_trans.iloc[0]
with gr.Blocks(theme='Glass') as demo:
    with gr.Row():
        trans_id = gr.Number(
            value=transaction.name,
            interactive=False,
            label='ID'
        )
        trans_date = gr.DateTime(
            value=datetime.combine(transaction['transaction_date'], datetime.min.time()),
            interactive=False,
            include_time=False,
            label='Transaction Date'
        )
    with gr.Row():
        trans_desc = gr.Textbox(
            value=transaction['description'],
            interactive=False,
            label='Description'
        )
        trans_amount = gr.Number(
            value=transaction['amount'],
            interactive=False,
            label='Amount'
        )

    with gr.Row():
        category = gr.Dropdown(
            choices=df_cat.parent_name.unique().tolist(),
            interactive=True,
            label='Category'
        )

        subcategory = gr.Dropdown(
            choices=[],
            interactive=True,
            allow_custom_value=True,
            label='Sub Category'
        )

        submit_box = gr.Checkbox(
            interactive=True,
            label='Submitted'
        )

    category.change(
        get_subcategories,
        inputs=category,
        outputs=subcategory
    )

    with gr.Row():
        previous_btn = gr.Button('Previous')
        previous_btn.click(fn=fetch_prev_transaction, inputs=trans_id, outputs=[trans_id, trans_date, trans_desc, trans_amount, submit_box])

        openai_btn = gr.Button('Call OpenAI')
        openai_btn.click(fn=openai_call, inputs=trans_id, outputs=[category, subcategory])

        submit_btn = gr.Button('Submit')
        submit_btn.click(fn=submit_cat, inputs=[trans_id, subcategory], outputs=[submit_box])

        next_btn = gr.Button('Next')
        next_btn.click(fn=fetch_next_transaction, inputs=trans_id, outputs=[trans_id, trans_date, trans_desc, trans_amount, submit_box])

demo.launch()

In [ ]:
df_labeled = (df_trans.query('category_name==category_name')
              .reset_index()
              .merge(df_cat.loc[:, ['category_name', 'category_id']], how='left', on='category_name')
              .set_index('id'))
df_labeled

with extractors.engine.begin() as conn:
    for idx, row in df_labeled.iterrows():
        conn.execute(
            text("UPDATE transactions SET category_id = :value WHERE id = :id"),
            {"value": int(row['category_id']), "id": idx}
        )